---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [231]:
def date_sorter():
    
    import pandas as pd

    doc = []

    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

    dates = pd.DataFrame()
    
    r1 = '(\d{1,2})[/-](\d{1,2})[/-](\d{2}|\d{4})[^\d]'
    r2 = '(\d{1,2})\s((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*\.?\s(\d{2,4})'
    r3 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[,.]?\s(\d{1,2})\,?\s(\d{2,4})'
    r4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[,.]?\s(\d{2,4})'
    r5 = '(\d{1,2})\/(\d{2,4})'
    r6 = '(\d{4})'
    
    data = df.str.extractall(r1).loc[:, [0, 1, 2]]

    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['mm', 'dd', 'yy']
    ex_dates.index = idx
    
    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]
    
    data = df.str.extractall(r2).loc[:, [1, 0, 2]]

    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['mm', 'dd', 'yy']
    ex_dates.mm = ex_dates.mm.map(months)
    ex_dates.index = idx

    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]
    
    data = df.str.extractall(r3).loc[:, [0, 1, 2]]

    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['mm', 'dd', 'yy']
    ex_dates.mm = ex_dates.mm.map(months)
    ex_dates.index = idx
    
    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]
    
    data = df.str.extractall(r4)
    
    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['mm', 'yy']
    ex_dates['dd'] = '01'
    ex_dates = ex_dates[['mm', 'dd', 'yy']]
    ex_dates.mm = ex_dates.mm.map(months)
    ex_dates.index = idx
    
    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]

    data = df.str.extractall(r5)
    
    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['mm', 'yy']
    ex_dates['dd'] = '01'
    ex_dates = ex_dates[['mm', 'dd', 'yy']]
    ex_dates.index = idx

    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]

    data = df.str.extractall(r6)
    
    idx = data.index.droplevel(1)
    ex_dates = data.reset_index(drop=True)
    
    ex_dates.columns = ['yy']
    ex_dates['dd'] = '01'
    ex_dates['mm'] = '01'
    ex_dates = ex_dates[['mm', 'dd', 'yy']]
    ex_dates.index = idx

    dates = dates.append(ex_dates)
    df = df[~df.index.isin(idx)]
    
    def change(row):
    
        if len(row['yy']) == 2:
            if row['yy'][0] == 0:
                row['yy'] = '20' + row['yy']
            else:
                row['yy'] = '19' + row['yy']

        if len(row['mm']) == 1:
            row['mm'] = '0' + row['mm']

        if len(row['dd']) == 1:
            row['dd'] = '0' + row['dd']
            
        return row

    dates = dates.apply(lambda x: change(x), axis=1)
    
    dates['date'] = dates['mm'] + '/' + dates['dd'] + '/' + dates['yy']
    dates = dates.drop(['mm', 'dd', 'yy'], axis=1)
    dates = dates.apply(pd.to_datetime).sort_values('date')
    s = pd.Series(dates.index)
    
    return s